# RxCadre Dataset Analysis

Our goal is to be able to extract information from the RxCadre datasets to mesh it with an external NASA dataset. This is an introductory notebook to give some foundation to this procedure.

## RxCadre Ignition Data

https://www.fs.usda.gov/rds/archive/catalog/RDS-2017-0065

This dataset actually describes how the ignition devices move so that we can test to see how we can join this table with the NASA one

In [6]:
# We will first be installing geopandas to be able to read the gdb files that come from RxCadre
!pip3 install geopandas

In [11]:
# import statements
# from google.colab import drive
# drive.mount('/content/drive')
import geopandas

In [12]:
# read in the gdb file provided by the database
data = geopandas.read_file("data.gdb") # make sure to update to your path
display(data.head())

,OBJECTID,LATITUDE,LONGITUDE,ALTITUDE,EASTING,NORTHING,UTCDATE,UTCTIME,SOG,COG,SATS_USED,HDOP,VDOP,PDOP,QUALITY,DIFF_AGE,geometry
0,1,30.525149,-86.736444,18.5,525284.544,3377008.024,2012-11-04T00:00:00+00:00,183122,0.1,141.800003,8,1.1,1.6,1.9,1,0.0,POINT Z (525284.544 3377008.024 18.500)
1,2,30.525151,-86.736447,18.4,525284.224,3377008.208,2012-11-04T00:00:00+00:00,183123,0.1,18.200001,8,1.1,1.6,1.9,2,1.0,POINT Z (525284.224 3377008.208 18.400)
2,3,30.525158,-86.736455,18.3,525283.423,3377008.945,2012-11-04T00:00:00+00:00,183124,2.1,304.100006,8,1.1,1.6,1.9,2,1.0,POINT Z (525283.423 3377008.944 18.300)
3,4,30.525164,-86.736469,18.1,525282.142,3377009.680,2012-11-04T00:00:00+00:00,183125,5.6,303.600006,8,1.1,1.6,1.9,2,2.0,POINT Z (525282.142 3377009.680 18.100)
4,5,30.525169,-86.736482,18.0,525280.861,3377010.231,2012-11-04T00:00:00+00:00,183126,4.9,301.000000,8,1.1,1.6,1.9,2,3.0,POINT Z (525280.861 3377010.231 18.000)


For some reason reading the gdb file like this only shows data from the 2012-11-04 dataset (L1G). So let's dive into this deeper and explore each gdbtable individually

In [25]:
import glob
import pandas as pd

path = "/data.gdb/*" # make sure to update to your path
dfs = {}
count = 0
booly = True
for file in glob.glob(path):
    if ".gdbtable" in file:
        gdbtable = geopandas.read_file(file)
        count += 1
        cols = gdbtable.columns
    if str(cols) in dfs:
        dfs[str(cols)].append(gdbtable)
    else:
        dfs[str(cols)] = [gdbtable]

print("Total number of tables:", count)

Total number of tables: 0


In [15]:
# Let's extract only the data with SOG (Speed over ground feet per second) for the ATVs in 2012
# I realized that in 2011, they only used aerial devices and it seems that the data of their locations is not available to us in the dataset. Therefore, let's focus on 2012
# for all intensive purposes

df_2012 = None
for key in dfs.keys():
    if 'SOG' in key:
        df_2012 = pd.concat(dfs[key])
        break
print("Number of rows:", len(df_2012))
df_2012.head() # check to see HHMMSS format

TypeError: object of type 'NoneType' has no len()

Okay, that seems like a lot let's see if we can now use the date and time to connect with another dataset

## NASA Aura Dataset (not working)

In [ ]:
!pip install pydap

     |████████████████████████████████| 2.3 MB 3.7 MB/s 
     |████████████████████████████████| 114 kB 68.9 MB/s 


In [ ]:
from pydap.client import open_url
from pydap.cas.urs import setup_session

username = ""
password = ""
dataset_url = "https://acdisc.gesdisc.eosdis.nasa.gov/data/Aura_OMI_Level3/OMHCHOd.003/2012/OMI-Aura_L3-OMHCHOd_2012m1108_v003-2019m0726t083354.nc"
session = setup_session(username, password, check_url=dataset_url)
dataset = open_url(dataset_url, session=session)


Exception: ignored

In [ ]:
import requests

with requests.Session() as session:

  # s.auth = (username, password)

  r1 = session.request('get', dataset_url)

  r = session.get(r1.url, auth=(username, password))

  if r.ok:

    print(r.content)

## Amemometer Data

In [6]:
# https://www.fs.usda.gov/rds/archive/catalog/RDS-2016-0038
amem = pd.read_csv("/content/drive/MyDrive/research/rxcadre/amemometer.csv", encoding = "ISO-8859-1")

In [7]:
amem.head()

,FileRecNo,Date,Time GMT+00:00,WindSpeed mph,WindGust mph,WindDirection Degrees_from_north,PlotID,InstrumentID,TagID,Latitude (Y),Longitude (X),Easting (X),Northing (Y),EllipsoidHt(Z)
0,1,11/03/12,08:35:35 PM,2.48,18.68,292.0,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51
1,2,11/03/12,08:35:38 PM,7.05,7.05,240.1,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51
2,3,11/03/12,08:35:41 PM,8.30,8.30,237.3,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51
3,4,11/03/12,08:35:44 PM,8.72,8.72,237.3,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51
4,5,11/03/12,08:35:47 PM,8.72,8.72,240.1,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51


In [8]:
# Let's hastily try to combine with the previous dataset. First we need to clean a little
import re

amem.columns = amem.columns.str.replace(" ", "")
amem['Date']= pd.to_datetime(amem['Date'], utc=True)
# amem['Time'] = pd.to_datetime(amem['TimeGMT+00:00'], utc=True)

def dms2dd(s):
  # example: s = """0°51'56.29"S"""
  degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
  dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
  if direction in ('S','W'):
      dd*= -1
  return dd

amem["LATITUDE"] = amem["Latitude(Y)"].apply(dms2dd)
amem["LONGITUDE"] = amem["Longitude(X)"].apply(dms2dd)

In [9]:
amem.head()

,FileRecNo,Date,TimeGMT+00:00,WindSpeedmph,WindGustmph,WindDirectionDegrees_from_north,PlotID,InstrumentID,TagID,Latitude(Y),Longitude(X),Easting(X),Northing(Y),EllipsoidHt(Z),LATITUDE,LONGITUDE
0,1,2012-11-03 00:00:00+00:00,08:35:35 PM,2.48,18.68,292.0,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51,30.529889,-85.261036
1,2,2012-11-03 00:00:00+00:00,08:35:38 PM,7.05,7.05,240.1,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51,30.529889,-85.261036
2,3,2012-11-03 00:00:00+00:00,08:35:41 PM,8.30,8.30,237.3,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51,30.529889,-85.261036
3,4,2012-11-03 00:00:00+00:00,08:35:44 PM,8.72,8.72,237.3,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51,30.529889,-85.261036
4,5,2012-11-03 00:00:00+00:00,08:35:47 PM,8.72,8.72,240.1,L1G,A2,21,"30°31'47.602""","-86°44'20.271""",525041.5,3377532.74,14.51,30.529889,-85.261036


In [10]:
devices_df = df_2012
devices_df["Date"] = pd.to_datetime(devices_df["UTCDATE"], utc=True)
# devices_df["Time"] = pd.to_datetime(devices_df["UTCTIME"], utc=True)

In [11]:
devices_df.head() # Exploring what the time is?

,OBJECTID,LATITUDE,LONGITUDE,ALTITUDE,EASTING,NORTHING,UTCDATE,UTCTIME,SOG,COG,SATS_USED,HDOP,VDOP,PDOP,QUALITY,DIFF_AGE,geometry,Date
0,1,30.525149,-86.736444,18.5,525284.544,3377008.024,2012-11-04T00:00:00+00:00,183122,0.1,141.800003,8,1.1,1.6,1.9,1,0.0,POINT Z (525284.544 3377008.024 18.500),2012-11-04 00:00:00+00:00
1,2,30.525151,-86.736447,18.4,525284.224,3377008.208,2012-11-04T00:00:00+00:00,183123,0.1,18.200001,8,1.1,1.6,1.9,2,1.0,POINT Z (525284.224 3377008.208 18.400),2012-11-04 00:00:00+00:00
2,3,30.525158,-86.736455,18.3,525283.423,3377008.945,2012-11-04T00:00:00+00:00,183124,2.1,304.100006,8,1.1,1.6,1.9,2,1.0,POINT Z (525283.423 3377008.944 18.300),2012-11-04 00:00:00+00:00
3,4,30.525164,-86.736469,18.1,525282.142,3377009.680,2012-11-04T00:00:00+00:00,183125,5.6,303.600006,8,1.1,1.6,1.9,2,2.0,POINT Z (525282.142 3377009.680 18.100),2012-11-04 00:00:00+00:00
4,5,30.525169,-86.736482,18.0,525280.861,3377010.231,2012-11-04T00:00:00+00:00,183126,4.9,301.000000,8,1.1,1.6,1.9,2,3.0,POINT Z (525280.861 3377010.231 18.000),2012-11-04 00:00:00+00:00


# Reading Fire Behavior Data

In [12]:
# https://www.fs.usda.gov/rds/archive/catalog/RDS-2016-0038
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/research/rxcadre/FireBehavior.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [13]:
# focus only on L1G and create big pandas
import os

xcels = []
for file in os.listdir("/content/FireBehavior/l1g"):
  # making this smaller for proof of concept optimization
  if len(xcels) > 2:
    break
  if "~$" in file:
    continue
  xcels.append(pd.read_excel("/content/FireBehavior/l1g/" + file, sheet_name="burn1"))

firebehavior = pd.concat(xcels)
firebehavior.head()

,Unnamed: 0,Date,UTC Time,Time After NAR Signal Detection (s),Temperature (degC),MT-total(kW/m^2),MT-rad(kW/m^2),Convective(kW/m^2),Total(kW/m^2),NAR(kW/m^2),...,Medtherm Body Temp,Vert. Wind(m/s),Horiz. Wind(m/s),Zeroed Vert. Wind(m/s),Zeroed Horiz. Wind(m/s),Battery,Vert. Wind 30 sec. avg.(m/s),Horiz. Wind 30 sec. avg.(m/s),Zeroed Vert. Wind 30 sec. avg.(m/s),Zeroed Horiz. Wind 30 sec. avg.(m/s)
0,0.1,2012-11-04,17:39:48.800000,0.0,26.57679,0.0,0.0,0.0,0.0,0.0,...,NAN,2.666134,2.811149,-0.403326,3.102748,12.34775,NaN,NaN,NaN,NaN
1,41027,2012-11-04,17:39:48.900000,0.1,26.57679,0.0,0.0,0.0,0.0,0.0,...,NAN,2.715232,2.026845,0.479193,2.562644,12.34227,NaN,NaN,NaN,NaN
2,42309,2012-11-04,17:39:49,0.2,26.57679,0.0,0.0,0.0,0.0,0.0,...,NAN,2.616603,2.165843,-0.781293,2.627834,12.34775,NaN,NaN,NaN,NaN
3,18:48:11.400000,2012-11-04,17:39:49.100000,0.3,26.57679,0.0,0.0,0.0,0.0,0.0,...,NAN,2.617048,2.490635,-0.781466,2.815762,12.34227,NaN,NaN,NaN,NaN
4,18:50:19.600000,2012-11-04,17:39:49.200000,0.4,26.57679,0.0,0.0,0.0,0.0,0.0,...,NAN,2.762049,2.810720,0.767959,3.102236,12.35322,NaN,NaN,NaN,NaN


In [14]:
rateOfSpread = firebehavior[["Date", "UTC Time", "Conv. 1 sec. avg.(kW/m^2)"]]
rateOfSpread.columns = ["date", "time", "rate"]
rateOfSpread = rateOfSpread.dropna()
rateOfSpread["date"] = [date.strftime('%m-%d-%Y') for date in list(rateOfSpread['date'])]
rateOfSpread["time"] = [time.strftime('%H:%M:%S.%f') for time in list(rateOfSpread['time'])]

rateOfSpread.head()

,date,time,rate
4,11-04-2012,17:39:49.200000,-0.023942
5,11-04-2012,17:39:49.300000,-0.025926
6,11-04-2012,17:39:49.400000,-0.027911
7,11-04-2012,17:39:49.500000,-0.029896
8,11-04-2012,17:39:49.600000,-0.031881


In [15]:
rateOfSpread["utc"] = rateOfSpread['date'] + ' ' + rateOfSpread['time']
rateOfSpread = rateOfSpread.drop(columns=["date", "time"])
rateOfSpread.head()

,rate,utc
4,-0.023942,11-04-2012 17:39:49.200000
5,-0.025926,11-04-2012 17:39:49.300000
6,-0.027911,11-04-2012 17:39:49.400000
7,-0.029896,11-04-2012 17:39:49.500000
8,-0.031881,11-04-2012 17:39:49.600000


# Combine Datasets

In [16]:
# Need to only focus on L1G for now
# https://www.fs.usda.gov/rds/archive/catalog/RDS-2016-0038
amem_l1g = pd.read_csv("/content/drive/MyDrive/research/rxcadre/amem_l1g.csv", encoding = "ISO-8859-1")
# print(amem_l1g.columns)
amem_l1g = amem_l1g[["_Date", "_Time_GMT_00_00", "_WindSpeed_mph", "_WindGust_mph", "_WindDirection_Degrees_from_nort"]]
amem_l1g.columns = ["date", "time", "windspeed", "windgust", "winddirection"]
amem_l1g.head()

,date,time,windspeed,windgust,winddirection
0,11/03/2012,20:35:35.000,2.48,18.68,292.0
1,11/03/2012,20:35:38.000,7.05,7.05,240.1
2,11/03/2012,20:35:41.000,8.30,8.30,237.3
3,11/03/2012,20:35:44.000,8.72,8.72,237.3
4,11/03/2012,20:35:47.000,8.72,8.72,240.1


In [17]:
amem_l1g["utc"] = amem_l1g['date'] + ' ' + amem_l1g['time']
amem_l1g = amem_l1g.drop(columns=["time", "date"])
amem_l1g.head()

,windspeed,windgust,winddirection,utc
0,2.48,18.68,292.0,11/03/2012 20:35:35.000
1,7.05,7.05,240.1,11/03/2012 20:35:38.000
2,8.30,8.30,237.3,11/03/2012 20:35:41.000
3,8.72,8.72,237.3,11/03/2012 20:35:44.000
4,8.72,8.72,240.1,11/03/2012 20:35:47.000


In [18]:
# convert utc columns to datetime
amem_l1g["utc"] = pd.to_datetime(amem_l1g["utc"])
rateOfSpread["utc"] = pd.to_datetime(rateOfSpread["utc"])

In [39]:
# We are setting the index
am = amem_l1g
ros = rateOfSpread
am = am.set_index("utc")
ros = ros.set_index("utc")
am = am.sort_index()
ros = ros.sort_index()

In [41]:
# now let's merge the two
merged = pd.merge_asof(am, ros, on="utc", tolerance=pd.Timedelta('5min'))
merged = merged.dropna()
merged

,utc,windspeed,windgust,winddirection,rate
1927333,2012-11-04 17:39:50,9.13,9.13,304.7,-0.037913
1927334,2012-11-04 17:39:50,8.30,8.30,320.1,-0.037913
1927335,2012-11-04 17:39:50,2.82,2.82,307.5,-0.037913
1927336,2012-11-04 17:39:50,6.22,6.22,353.8,-0.037913
1927337,2012-11-04 17:39:50,7.05,7.05,278.0,-0.037913
...,...,...,...,...,...
2064970,2012-11-04 19:09:47,9.55,9.55,248.5,-0.084964
2064971,2012-11-04 19:09:47,14.94,14.94,296.2,-0.084964
2064972,2012-11-04 19:09:47,7.47,7.47,318.7,-0.084964
2064973,2012-11-04 19:09:47,8.72,8.72,292.0,-0.084964


# Basic DL Model

In [42]:
import torch as th

# convert variables to PyTorch tensor 
X1 = th.tensor(merged[['windspeed']].values)
X2 = th.tensor(merged[["windgust"]].values)
X3 = th.tensor(merged[["winddirection"]].values)
X = th.stack([X1, X2, X3], 2)
y = th.tensor(merged[['rate']].values, dtype=th.float32)

In [43]:
in_features = 3 # number of independent variables
out_features = 1 # dimension of predicted variables
# bias is default true and can be skipped
model = th.nn.Linear(in_features=in_features, out_features=out_features, bias=True)

In [44]:
mse_loss = th.nn.MSELoss()

In [45]:
optimizer = th.optim.SGD(model.parameters(), lr=0.002)

In [ ]:
# set epochs
n_epoch = 10
for i in range(n_epoch):
    # predict model with current regression parameters
    # forward pass (feed the data to model)
    y_pred = model(X.float())
    # calculate loss function
    step_loss = mse_loss(y_pred, y)
        
    # Backward to find the derivatives of the loss function with respect to regression parameters
    # make any stored gradients to zero
    # backward pass (go back and update the regression parameters to minimize the loss)
    optimizer.zero_grad()
    step_loss.backward()
    # update with current step regression parameters 
    optimizer.step()
    print ('epoch [{}], Loss: {:.2f}'.format(i, step_loss.item()))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([137642, 1])) that is different to the input size (torch.Size([137642, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
# ACtion items
# share the code with meng students
# run this locally if doesnt work
# combine with Rohans